In [1]:
#Getting data from a PDF and putting in a dataframe
import PyPDF2
import pandas as pd
file=open('Task Request_Data Analytics Team.pdf','rb')
pdf_reader=PyPDF2.PdfFileReader(file)
page=1
rows_list=[]
for pageno in range(pdf_reader.numPages):
    row=1
    read_page=pdf_reader.getPage(pageno)
    page_data=read_page.extractText()
    page_list=page_data.split('\n')
    for data in page_list:
        data=data.strip()
        rows_list.append([page,row,data])
        row=row+1
    #Getting only the sentence containg words and ignoring sentence containg PageNo. detail in PDF
    rows_list=rows_list[:-5]
    page=page+1    
df = pd.DataFrame(rows_list,columns = ['Page No.','Line No.','Line'])
df['Line']=df['Line'].apply(lambda x: x.strip())
df.head()

,Page No.,Line No.,Line
0,1,1,BACKGROUND
1,1,2,This request is for a startup company Environm...
2,1,3,that does data analytics for various companies...
3,1,4,and buildings sector. EBC is new and will be b...
4,1,5,companies with environment compliance in this ...


In [2]:
#Creating dataframe with word, page number, line number,wordlength from exsisting fdataframe
finalword_list=[]
page_list=df['Page No.'].tolist()
number_list=df['Line No.'].tolist()
line_list=df['Line'].tolist()
# To get only word rather than a combination of below with word
stop_list=['¥',',','(',')','.',':','!']
total_lines=len(line_list)
for lineno in range(total_lines):
    page=page_list[lineno]
    number=number_list[lineno]
    line=line_list[lineno]
    word_list=line.split()
    for word in word_list:
        append_list=[]
        if ((word[0] in stop_list) and (word[-1] in stop_list)):
            data=word[1:-1].strip()
        elif (word[0] in stop_list):
            data=word[1:].strip()
        elif (word[-1] in stop_list):
            data=word[:-1].strip()
        else:
            flag=0
            splitter=''
            #For words like B.Significance 
            for sign in stop_list:
                if (sign in word):
                    flag=1
                    splitter=sign
                    break
                else:
                    continue
            if flag==0:
                data=word.strip()
            else:
                data=word.split(splitter)[1].strip()
        length=len(data)
        append_list.append(page)
        append_list.append(number)
        append_list.append(data)            
        append_list.append(length)
        finalword_list.append(append_list)
word_frame= pd.DataFrame(finalword_list,columns = ['Page No.','Line No.','Word','WordLength'])

In [3]:
#Frame containing PageNo., LineNo.,Word, Wordlength
word_frame.head()

,Page No.,Line No.,Word,WordLength
0,1,1,BACKGROUND,10
1,1,2,This,4
2,1,2,request,7
3,1,2,is,2
4,1,2,for,3


In [4]:
#List of strings given as input for which its location in Page and LineNo. to be displayed
key_tags=['for','these','services','company']
for word in key_tags:
    print('\n')
    print(word)
    print('Page','\t','LineNo.')
    for index,row in word_frame[word_frame['Word']==word].iterrows():
        print(row['Page No.'],'\t', row['Line No.'])



for
Page 	 LineNo.
1 	 2
1 	 3
1 	 10
1 	 23
2 	 16
3 	 2
3 	 13
3 	 15
3 	 23


these
Page 	 LineNo.
1 	 14
1 	 17


services
Page 	 LineNo.
1 	 5
1 	 14


company
Page 	 LineNo.
1 	 2
1 	 11
1 	 12
1 	 17
1 	 18
3 	 7


In [5]:
#Displaying PageNo., LineNo. where all words are in capital
print('Page and line where all words are in caps:')
print('PageNo.','\t','Line No.')
for index,row in df.iterrows():
    flag=0
    check_list=row['Line'].split()
    for word in check_list:
        if word.isupper()!=True:
            flag=1
            break
        else:
            continue
    if flag==0:
        print(row['Page No.'],'\t','\t',row['Line No.'])
    else:
        continue

Page and line where all words are in caps:
PageNo. 	 Line No.
1 	 	 1
1 	 	 32
3 	 	 1


In [6]:
#Creating and displaying line gram from data-frame containing all the words
word_gram=[]
for index, row in word_frame.iterrows():
    if row['Word'].lower() in word_gram:
        continue
    else:
        word_gram.append(row['Word'].lower())
#Printing all the word in the wordgram
for word in word_gram:
    print(word)

background
this
request
is
for
a
startup
company
environment
blueprint
consulting
ebc
that
does
data
analytics
various
companies
in
the
industrial
and
property
buildings
sector
new
will
be
based
uae
to
support
with
compliance
region
main
services
offered
includes
environmental
baseline
assessment
noise
air
soil
&
groundwater
reporting
2.auditing
monitoring
usually
submitting
local
authority
approval
then
issued
permit
which
they
need
comply
needs
demonstrate
through
auditing
traditionally
these
have
been
commissioned
separately
sits
consultant
hard
drive
submission
of
reports
are
done
copy
i
developed
disrupt
way
assessments
undertaken
once
engaged
by
collect
deliver
would
like
input
into
tableau
dashboard
where
client
access
received
has
uses
add
regular
on
top
it
so
as
see
trends
forming
made
easier
while
technical
expertise
hire
document
outlines
tasks
deliverables
form
role
team
consist
one
analyst
scientist
friday
14
august
2020
rev
0
page
13task
brief
date
roles
proper
collection

In [7]:
#Finding and displaying least occuring words
word_less=list(word_frame['Word'].value_counts().sort_values(ascending=True)[:10].index)
print('Least occuring words are:')
for word in word_less:
    print(word)

Least occuring words are:
infrastructure
UAE
authority
make
alienating
startup
training
developed
collect
where


In [8]:
#Marking two rows below where 10 least common words are found and three rows above the same
print('Printing two rows below and three rows above the least occuring word is found ')
print('\n')
for word in word_less:
    print('\n')
    print(word)
    print('\n')
    print('PageNo','\t','LineNo.')
    pages=list(word_frame[word_frame['Word']==word]['Page No.'])
    lines=list(word_frame[word_frame['Word']==word]['Line No.'])
    count_in_frame=len(pages)
    for i in range(count_in_frame):
        page=pages[i]
        line=lines[i]
        tranframe=df[df['Page No.']==page]
        index=tranframe[tranframe['Line No.']==4].index
        print(int(tranframe.loc[index-1]['Page No.']),'\t',int(tranframe.loc[index-1]['Line No.']))
        print(int(tranframe.loc[index-2]['Page No.']),'\t',int(tranframe.loc[index-2]['Line No.']))
        print(int(tranframe.loc[index+1]['Page No.']),'\t',int(tranframe.loc[index+1]['Line No.']))
        print(int(tranframe.loc[index+2]['Page No.']),'\t',int(tranframe.loc[index+2]['Line No.'])) 
        print(int(tranframe.loc[index+3]['Page No.']),'\t',int(tranframe.loc[index+3]['Line No.']))

Printing two rows below and three rows above the least occuring word is found 




infrastructure


PageNo 	 LineNo.
3 	 3
3 	 2
3 	 5
3 	 6
3 	 7


UAE


PageNo 	 LineNo.
1 	 3
1 	 2
1 	 5
1 	 6
1 	 7


authority


PageNo 	 LineNo.
1 	 3
1 	 2
1 	 5
1 	 6
1 	 7


make


PageNo 	 LineNo.
2 	 3
2 	 2
2 	 5
2 	 6
2 	 7


alienating


PageNo 	 LineNo.
2 	 3
2 	 2
2 	 5
2 	 6
2 	 7


startup


PageNo 	 LineNo.
1 	 3
1 	 2
1 	 5
1 	 6
1 	 7


training


PageNo 	 LineNo.
2 	 3
2 	 2
2 	 5
2 	 6
2 	 7


developed


PageNo 	 LineNo.
1 	 3
1 	 2
1 	 5
1 	 6
1 	 7


collect


PageNo 	 LineNo.
1 	 3
1 	 2
1 	 5
1 	 6
1 	 7


where


PageNo 	 LineNo.
1 	 3
1 	 2
1 	 5
1 	 6
1 	 7
